# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

# Flow:
- Take dollar exchange rate
- Take euro exchange rate
- Take gold exchange rate
- Import data and update
- Compute new prices for products

## Need to install:
!py -m pip install selenium

download [ChromeDriver](https://chromedriver.chromium.org/downloads)

### Buscando as informações

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [2]:
browser = webdriver.Chrome() # webdriver.Chrome('./chromedriver.exe') if chromedriver is on root
def get_currency_exchange_rate(by:str):
    browser.get('https://www.google.com.br/')
    
    search_input_xpath = '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div[2]/div[2]/input'
    browser.find_element(By.XPATH, search_input_xpath).send_keys(f'cotação {by}', Keys.ENTER)
    
    value_xpath = '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'
    currency_exchange_rate = browser.find_element(By.XPATH, value_xpath).get_attribute('data-value')
    
    return currency_exchange_rate


def get_gold_exchange_rate():
    browser.get('https://www.melhorcambio.com/ouro-hoje')
    
    value_xpath = '//*[@id="comercial"]'
    gold_exchange_rate = browser.find_element(By.XPATH, value_xpath).get_attribute('value')
    gold_exchange_rate = gold_exchange_rate.replace(',', '.')
    return gold_exchange_rate

dollar = float(get_currency_exchange_rate(by='dólar'))
euro = float(get_currency_exchange_rate(by='euro'))
gold = float(get_gold_exchange_rate())
browser.quit()

### Agora vamos atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [3]:
import pandas as pd

In [4]:
products = pd.read_excel('./Produtos.xlsx')
display(products)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [5]:
currencies = [
    {'name': 'Dólar', 'value': dollar},
    {'name': 'Euro', 'value': euro},
    {'name': 'Ouro', 'value': gold}
]
for currency in currencies:
    products.loc[products['Moeda'] == currency['name'], 'Cotação'] = currency['value']

products['Preço de Compra'] = products['Preço Original'] * products['Cotação']

products['Preço de Venda'] = products['Preço de Compra'] * products['Margem']

display(products)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.534600,5534.544654,1.40,7748.362516
1,Carro Renault,4500.00,Euro,6.333326,28499.966096,2.00,56999.932191
2,Notebook Dell,899.99,Dólar,5.534600,4981.084654,1.70,8467.843912
3,IPhone,799.00,Dólar,5.534600,4422.145400,1.70,7517.647180
4,Carro Fiat,3000.00,Euro,6.333326,18999.977397,1.90,36099.957054
5,Celular Xiaomi,480.48,Dólar,5.534600,2659.264608,2.00,5318.529216
6,Joia 20g,20.00,Ouro,324.800000,6496.000000,1.15,7470.400000


### Agora vamos exportar a nova base de preços atualizada

In [6]:
products.to_excel('ProdutosUpdated.xlsx', index=False)